<a href="https://colab.research.google.com/github/cai4cai/torchsparsegradutils/blob/test-notebook/tests/spd_forward_solve_notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# A basic notebook to compare different solvers for a SPD matrix

First import all necessary modules.

In [1]:
import torch
print(f'Running PyTorch version: {torch.__version__}')

torchdevice = torch.device('cpu')
if torch.cuda.is_available():
  torchdevice = torch.device('cuda')
  print('Default GPU is ' + torch.cuda.get_device_name(torch.device('cuda')))
print('Running on ' + str(torchdevice))

import numpy as np
import scipy.io
import scipy.sparse.linalg

import cupyx
import cupyx.scipy.sparse as csp

import jax
from jax.config import config
config.update("jax_enable_x64", True)

!pip install git+https://github.com/cai4cai/torchsparsegradutils
import torchsparsegradutils as tsgu
import torchsparsegradutils.utils
import torchsparsegradutils.cupy as tsgucupy
import torchsparsegradutils.jax as tsgujax

import time
import urllib
import os.path
import tarfile

Running PyTorch version: 1.13.0+cu116
Default GPU is Tesla T4
Running on cuda
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/cai4cai/torchsparsegradutils to /tmp/pip-req-build-plwawj9f
  Running command git clone -q https://github.com/cai4cai/torchsparsegradutils /tmp/pip-req-build-plwawj9f
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done


Now load an example SPD matrix and create a random RHS vector

In [2]:
def load_mat_from_suitesparse_collection(dirname,matname):
  base_url = 'https://suitesparse-collection-website.herokuapp.com/MM/'
  url = base_url + dirname + '/' + matname + '.tar.gz'
  compressedlocalfile = matname + '.tar.gz'
  if not os.path.exists(compressedlocalfile):
    print(f'Downloading {url}')
    urllib.request.urlretrieve(url, filename=compressedlocalfile)

  localfile = './' + matname + '/' + matname + '.mtx'
  if not os.path.exists(localfile):
    print(f'untarring {compressedlocalfile}')
    srctarfile = tarfile.open(compressedlocalfile)
    srctarfile.extractall('./')
    srctarfile.close()

  A_np_coo = scipy.io.mmread(localfile)
  print(f'Loaded suitesparse matrix {dirname}/{matname}: type={type(A_np_coo)}, shape={A_np_coo.shape}')
  return A_np_coo

A_np_coo = load_mat_from_suitesparse_collection('Rothberg','cfd2')
A_np_csr = scipy.sparse.csr_matrix(A_np_coo)

b_np = np.random.randn(A_np_coo.shape[1])
print(f'Created random RHS with shape={b_np.shape}')

Loaded suitesparse matrix Rothberg/cfd2: type=<class 'scipy.sparse.coo.coo_matrix'>, shape=(123440, 123440)
Created random RHS with shape=(123440,)


Define some helper functions to run the tests

In [3]:
def scipy_test(A, b, tested_solver, print_string):
  t = time.time()
  x = tested_solver(A, b)
  elapsed = time.time() - t
  resnorm = scipy.linalg.norm(A @ x - b)
  print(f'{print_string} took {elapsed:.2f} seconds - resnorm={resnorm:.2e}')
  return elapsed, resnorm

Run the tests with the scipy routines (it can take a while)

In [4]:
run_scipy = True
if run_scipy:
  t, n = scipy_test(A_np_coo, b_np, scipy.sparse.linalg.spsolve, 'scipy.spsolve COO')
  t, n = scipy_test(A_np_csr, b_np, scipy.sparse.linalg.spsolve, 'scipy.spsolve CSR')
  t, n = scipy_test(A_np_coo, b_np, lambda A, b: scipy.sparse.linalg.cg(A,b)[0], 'scipy.cg COO')
  t, n = scipy_test(A_np_csr, b_np, lambda A, b: scipy.sparse.linalg.cg(A,b)[0], 'scipy.cg CSR')
  t, n = scipy_test(A_np_coo, b_np, lambda A, b: scipy.sparse.linalg.bicgstab(A,b)[0], 'scipy.bicgstab COO')
  t, n = scipy_test(A_np_csr, b_np, lambda A, b: scipy.sparse.linalg.bicgstab(A,b)[0], 'scipy.bicgstab CSR')
  t, n = scipy_test(A_np_coo, b_np, lambda A, b: scipy.sparse.linalg.minres(A,b)[0], 'scipy.minres COO')
  t, n = scipy_test(A_np_csr, b_np, lambda A, b: scipy.sparse.linalg.minres(A,b)[0], 'scipy.minres CSR')
else:
  print('Skipping scipy tests')

/usr/local/lib/python3.8/dist-packages/scipy/sparse/linalg/dsolve/linsolve.py:144: SparseEfficiencyWarning: spsolve requires A be CSC or CSR matrix format
  warn('spsolve requires A be CSC or CSR matrix format',


scipy.spsolve COO took 108.62 seconds - resnorm=2.15e-10
scipy.spsolve CSR took 98.29 seconds - resnorm=2.49e-10
scipy.cg COO took 72.09 seconds - resnorm=3.50e-03
scipy.cg CSR took 55.59 seconds - resnorm=3.50e-03
scipy.bicgstab COO took 112.65 seconds - resnorm=2.31e-03
scipy.bicgstab CSR took 88.79 seconds - resnorm=2.31e-03
scipy.minres COO took 0.54 seconds - resnorm=3.05e+01
scipy.minres CSR took 0.38 seconds - resnorm=3.05e+01


Create corresponding PyTorch tensors

In [5]:
A_tcpu_csr = tsgucupy.c2t_csr(A_np_csr)
b_tcpu = torch.from_numpy(b_np)

if torch.cuda.is_available():
  A_tgpu_csr = A_tcpu_csr.to(torchdevice)
  b_tgpu = b_tcpu.to(torchdevice)

/usr/local/lib/python3.8/dist-packages/torchsparsegradutils/cupy/cupy_bindings.py:52: UserWarning: Sparse CSR tensor support is in beta state. If you miss a functionality in the sparse tensor support, please submit a feature request to https://github.com/pytorch/pytorch/issues. (Triggered internally at ../aten/src/ATen/SparseCsrTensorImpl.cpp:54.)
  x_torch = torch.sparse_csr_tensor(ind_ptr_t, idices_t, data_t, x_cupy.shape)


Define some helper function to run the tests with pytorch

In [6]:
def torch_test(A, b, tested_solver, print_string):
  t = time.time()
  x = tested_solver(A, b)
  elapsed = time.time() - t
  resnorm = torch.norm(A @ x - b).cpu().numpy()
  print(f'{print_string} took {elapsed:.2f} seconds - resnorm={resnorm:.2e}')
  print(f'GPU memory allocated: {torch.cuda.memory_allocated(device=torchdevice)/10**9:.2f}Gb'
    f' - max allocated: {torch.cuda.max_memory_allocated(device=torchdevice)/10**9:.2f}Gb')
  #print(torch.cuda.memory_summary(abbreviated=True))
  return elapsed, resnorm

Run the tests with our pytorch routines (this can take a while)

In [7]:
print(f'GPU memory allocated: {torch.cuda.memory_allocated(device=torchdevice)/10**9:.2f}Gb'
  f' - max allocated: {torch.cuda.max_memory_allocated(device=torchdevice)/10**9:.2f}Gb')
#print(torch.cuda.memory_summary(abbreviated=True))

#t, n = torch_test(A_tcpu_csr, b_tcpu, tsgu.sparse_generic_solve, 'tsgu.sparse_generic_solve CPU CSR')
if torch.cuda.is_available():
  t, n = torch_test(A_tgpu_csr, b_tgpu, tsgu.sparse_generic_solve, 'tsgu.sparse_generic_solve GPU CSR')

#t, n = torch_test(A_tcpu_csr, b_tcpu, tsgu.sparse_generic_lstsq, 'tsgu.sparse_generic_lstsq CPU CSR')
#if torch.cuda.is_available():
#  t, n = torch_test(A_tgpu_csr, b_tgpu, tsgu.sparse_generic_lstsq, 'tsgu.sparse_generic_lstsq GPU CSR')

mysolver = lambda A, b: tsgu.sparse_generic_solve(A,b,solve=tsgu.utils.minres)
if torch.cuda.is_available():
  t, n = torch_test(A_tgpu_csr, b_tgpu, mysolver, 'tsgu.sparse_generic_solve minres GPU CSR')

mysolver = lambda A, b: tsgu.sparse_generic_solve(A,b,solve=tsgu.utils.linear_cg)
if torch.cuda.is_available():
  t, n = torch_test(A_tgpu_csr, b_tgpu, mysolver, 'tsgu.sparse_generic_solve cg GPU CSR')

mysolver = lambda A, b: tsgu.sparse_generic_solve(A,b,solve=tsgu.utils.bicgstab)
if torch.cuda.is_available():
  t, n = torch_test(A_tgpu_csr, b_tgpu, mysolver, 'tsgu.sparse_generic_solve bicgstab GPU CSR')

jaxsolver = None
mysolver = lambda A, b: tsgujax.sparse_solve_j4t(A,b)
if torch.cuda.is_available():
  t, n = torch_test(A_tgpu_csr, b_tgpu, mysolver, 'tsgu.sparse_solve_j4t cg GPU CSR')

#cpsolver = lambda AA, BB: csp.linalg.cg(AA,BB)[0]
#mysolver = lambda A, b: tsgucupy.sparse_solve_c4t(A,b,solve=cpsolver)
#mysolver = lambda A, b: tsgucupy.sparse_solve_c4t(A,b)
#t, n = torch_test(A_tcpu_csr, b_tcpu, mysolver, 'tsgu.sparse_solve_c4t cg CPU CSR')
#if torch.cuda.is_available():
#  t, n = torch_test(A_tgpu_csr, b_tgpu, mysolver, 'tsgu.sparse_solve_c4t cg GPU CSR')

GPU memory allocated: 0.04Gb - max allocated: 0.04Gb
tsgu.sparse_generic_solve GPU CSR took 0.62 seconds - resnorm=1.93e+00
GPU memory allocated: 0.04Gb - max allocated: 0.05Gb
tsgu.sparse_generic_solve minres GPU CSR took 0.46 seconds - resnorm=1.93e+00
GPU memory allocated: 0.04Gb - max allocated: 0.05Gb
tsgu.sparse_generic_solve cg GPU CSR took 0.06 seconds - resnorm=2.43e+02
GPU memory allocated: 0.04Gb - max allocated: 0.05Gb
tsgu.sparse_generic_solve bicgstab GPU CSR took 9.35 seconds - resnorm=2.60e-04
GPU memory allocated: 0.04Gb - max allocated: 0.05Gb
tsgu.sparse_solve_j4t cg GPU CSR took 6.39 seconds - resnorm=2.74e-03
GPU memory allocated: 0.04Gb - max allocated: 0.05Gb
